In [ ]:
import os, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from keras.layers import Dense,Flatten,Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam



In [ ]:


img_rows=28
img_cols=28
channels=1

img_shape = (img_rows,img_cols,channels)

zdim=100




In [ ]:
train_data = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_train.csv')
X_train = train_data.drop('label',axis=1)
X_train=X_train.values
X_train=X_train.reshape(-1,28,28,1)
print(X_train.shape)

In [ ]:
fig,axe=plt.subplots(2,2)
idx = 0
for i in range(2):
    for j in range(2):
        axe[i,j].imshow(X_train[idx].reshape(28,28),cmap='gray')
        idx+=1

In [ ]:
X_train =  X_train.astype('float32')
X_train = X_train/255
X_train = X_train*2 - 1.

In [ ]:
def build_gen(img_shape,zdim):
    model = Sequential()
    model.add(Dense(128,input_dim=zdim))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(28*28*1,activation='tanh'))
    model.add(Reshape(img_shape))
    return model

def build_dis(img_shape):
    model=Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

def build_gan(g_model,d_model):
    d_model.trainable=False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(lr=0.0002, beta_1=0.5)   
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
losses=[]
accuracies=[]
iteration_checks=[]

In [ ]:
def train(g_model,d_model,gan_model, iterations,batch_size,interval):

    real = np.ones((batch_size,1))
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        ids = np.random.randint(0,X_train.shape[0],batch_size)
        imgs = X_train[ids]

        z=np.random.normal(0,1,(batch_size,100))
        gen_imgs = g_model.predict(z)

        dloss_real = d_model.train_on_batch(imgs,real)
        dloss_fake = d_model.train_on_batch(gen_imgs, fake)

        dloss,accuracy = 0.5 * np.add(dloss_real,dloss_fake)

        z = np.random.normal(0, 1, (batch_size, 100))
        gloss = gan_model.train_on_batch(z,real)

        if (iteration+1) % interval == 0:
            losses.append((dloss,gloss))
            accuracies.append(100.0*accuracy)
            iteration_checks.append(iteration+1)

            print("%d [D loss: %f , acc: %.2f] [G loss: %f]" %
                  (iteration+1,dloss,100.0*accuracy,gloss))
            show_images(g_model)

In [ ]:
def show_images(gen):
    z = np.random.normal(0, 1, (16, 100))
    gen_imgs = gen.predict(z)
    gen_imgs = 0.5*gen_imgs + 0.5

    fig,axs = plt.subplots(4,4,figsize=(4,4),sharey=True,sharex=True)

    cnt=0
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
            axs[i, j].axis('off')
            cnt+=1

    fig.show()

In [ ]:
# create the discriminator
d_model = build_dis(img_shape)

# create the generator
g_model = build_gen(img_shape,zdim)


# create the gan
gan_model = build_gan(g_model,d_model)


train(g_model,d_model, gan_model, 5000,128,1000)